In [1]:
from dataset_preprocessing import ProductionModeDataset

import torch
from sklearn.inspection import permutation_importance
import numpy as np
import uproot
import torch.nn as nn
from matplotlib import pyplot as plt

from Classifier_module import Classifier
from torch.autograd import Variable


In [135]:
################# THIS IS WHERE YOU CHOOSE WHAT TO LOAD ################################
three_layers = False

path_to_model = "./models/twoLayerModel_weights_gpu_emu0"

event_type = "emu"  # could be ee, emu, mumu
root_path = "/depot-new/cms/top/mcnama20/TopSpinCorr-Run2-Entanglement/CMSSW_10_2_22/src/TopAnalysis/Configuration/analysis/diLeptonic/three_files/Nominal"

file = root_path + "/" + event_type + "_modified_root_1.root"

In [136]:
if not three_layers:
    # model = 0
    model = Classifier()
    model.load_state_dict(torch.load(path_to_model))
#     model.eval()

In [137]:



data_object = ProductionModeDataset(file)
x = data_object.get_eval_data()

['lb_delta_eta', 'lbbar_delta_eta', 'lnu_delta_eta', 'lnubar_delta_eta', 'lbarb_delta_eta', 'lbarbbar_delta_eta', 'lbarnu_delta_eta', 'lbarnubar_delta_eta', 'bnu_delta_eta', 'bnubar_delta_eta', 'bbarnu_delta_eta', 'bbarnubar_delta_eta', 'lb_delta_phi', 'lbbar_delta_phi', 'lnu_delta_phi', 'lnubar_delta_phi', 'lbarb_delta_phi', 'lbarbbar_delta_phi', 'lbarnu_delta_phi', 'lbarnubar_delta_phi', 'bnu_delta_phi', 'bnubar_delta_phi', 'bbarnu_delta_phi', 'bbarnubar_delta_phi', 'wplusb_delta_eta', 'wplusbbar_delta_eta', 'wminusb_delta_eta', 'wminusbbar_delta_eta', 'wplusb_delta_phi', 'wplusbbar_delta_phi', 'wminusb_delta_phi', 'wminusbbar_delta_phi', 'top_eta', 'top_boosted_eta', 'tbar_eta', 'tbar_boosted_eta', 'ttbar_delta_eta', 'ttbar_eta', 'llbar_delta_eta', 'bbbar_delta_eta', 'nunubar_delta_eta', 'top_phi', 'tbar_phi', 'ttbar_phi', 'ttbar_delta_phi', 'llbar_phi', 'llbar_delta_phi', 'bbbar_phi', 'bbbar_delta_phi', 'nunubar_phi', 'nunubar_delta_phi', 'l_eta', 'lbar_eta', 'l_phi', 'lbar_phi', '

In [138]:
weight = x[:,81]
target = x[:,80]
target = Variable(torch.from_numpy(target).type(torch.LongTensor))
y = np.transpose(x)
y = np.delete(y, [80, 81, 82], 0)
y = np.transpose(y)
input = Variable(torch.from_numpy(y).type(torch.Tensor).requires_grad_(True))

In [139]:
input.requires_grad=True

In [140]:
# optimizer = torch.optim.Adam(model.parameters(), , betas=(opt.b1, opt.b2)) 
# criterion = nn.MSELoss()   # mean squared error loss
criterion = nn.CrossEntropyLoss()


In [141]:
# model.retain_grad()
input.retain_grad()
output = model(input).requires_grad_(True)
output.retain_grad()

loss = criterion(output, target)
loss.retain_grad()
print(loss)

# external_grad = torch.tensor(1.0)
loss.backward()

tensor(0.9015, grad_fn=<NllLossBackward>)


In [142]:
print(input.grad)

tensor([[ 9.0678e-05, -2.1714e-04, -5.7967e-05,  ..., -4.3832e-06,
          6.8204e-04, -1.2712e-04],
        [ 3.8705e-05,  4.7423e-05, -1.9404e-05,  ...,  2.2734e-04,
          5.8973e-05, -3.5816e-05],
        [ 1.6685e-04, -4.7880e-05, -1.8559e-04,  ..., -1.3004e-04,
          9.0444e-05, -6.7031e-05],
        ...,
        [ 1.4680e-05,  1.2701e-04, -3.6132e-05,  ..., -4.4586e-05,
          3.0181e-04, -3.3041e-05],
        [-6.2596e-06,  5.7693e-06,  2.4699e-06,  ...,  5.4572e-05,
          4.6389e-05,  4.8701e-05],
        [ 6.0135e-05, -2.5962e-04, -1.4341e-04,  ...,  9.2977e-05,
          2.6396e-04,  5.4426e-04]])


In [143]:
mean_grad = input.grad.detach().numpy().mean(axis=0)
max_grad = np.max(mean_grad)

mean_grad = mean_grad/max_grad
mean_grad

array([ 8.07799697e-02,  6.08307868e-02,  2.36199889e-02,  1.17350258e-01,
        8.25684890e-03,  2.90746638e-03,  1.98056549e-01, -3.61698195e-02,
        1.15328461e-01,  6.32961392e-02, -9.58944783e-02, -1.31271956e-02,
        5.44626079e-02,  4.53951359e-02,  3.84598784e-02,  2.92467643e-02,
       -3.58630382e-02,  5.55347539e-02,  2.02903748e-02, -5.27100004e-02,
        2.47267028e-03, -2.18687411e-02,  2.32141614e-02,  2.09688209e-02,
       -9.49673355e-02, -1.44977607e-02, -8.92608223e-05,  7.57536814e-02,
        1.08153902e-01, -3.90725099e-02,  4.43832651e-02,  2.09163912e-02,
       -1.87097881e-02,  3.44761238e-02, -2.71196216e-02, -4.93810363e-02,
        1.46099135e-01,  1.35774082e-02, -1.02312148e-01, -7.63935298e-02,
        3.22180567e-03, -1.21727604e-02, -1.76235549e-02, -1.40019869e-02,
       -7.47243837e-02, -3.22282873e-02, -8.55048671e-02, -5.21382142e-04,
        2.38251593e-02, -1.25365416e-02, -2.41622664e-02,  7.74827898e-02,
       -6.86194524e-02,  

In [144]:
std_grad = np.std(input.grad.detach().numpy(), axis=0)/np.sqrt(input.shape[0])
std_grad

array([9.1065567e-07, 8.1202893e-07, 9.8942894e-07, 1.1065603e-06,
       8.0206877e-07, 9.2316031e-07, 1.2576696e-06, 1.0391935e-06,
       9.0280059e-07, 1.1411280e-06, 1.0697798e-06, 1.0467691e-06,
       6.3639334e-07, 6.7825852e-07, 7.5418626e-07, 6.9675167e-07,
       7.0774558e-07, 6.7716604e-07, 6.7315813e-07, 6.7749534e-07,
       6.7338954e-07, 6.6089416e-07, 6.6368540e-07, 7.0518615e-07,
       1.0184290e-06, 1.0781133e-06, 1.0762193e-06, 1.1403954e-06,
       7.0537777e-07, 7.3979214e-07, 7.1123253e-07, 7.4720919e-07,
       5.7082502e-07, 7.1426604e-07, 7.2603007e-07, 5.3379171e-07,
       1.2088718e-06, 7.3698391e-07, 8.8301948e-07, 7.9955612e-07,
       1.1366321e-06, 6.0906507e-07, 6.7006226e-07, 6.9895020e-07,
       6.8913226e-07, 6.7390880e-07, 6.8861601e-07, 5.3892688e-07,
       5.6102931e-07, 5.5939228e-07, 5.6279151e-07, 6.6008192e-07,
       6.2329207e-07, 6.6914185e-07, 6.1991608e-07, 6.3373074e-07,
       6.0758737e-07, 5.9478253e-07, 6.4397182e-07, 6.7121078e

In [145]:
events = uproot.open(file)
key = events.keys()[0]
input_variables = events[key].keys()

In [146]:
for i in mean_grad.argsort()[::-1]:
#     if mean_grad[i] - 2 * r.importances_std[i] > 0:
    print(f"{input_variables[i]:<8}                    "
          f"{mean_grad[i]:.3f}"
          f" +/- {std_grad[i]:.3f}")


ttbar_pt                    1.000 +/- 0.000
ttbar_boosted_pt                    0.450 +/- 0.000
nubar_pt                    0.405 +/- 0.000
b_pt                        0.230 +/- 0.000
l_pt                        0.201 +/- 0.000
lbarnu_delta_eta                    0.198 +/- 0.000
wplus_pt                    0.194 +/- 0.000
ttbar_delta_eta                    0.146 +/- 0.000
bbar_pt                     0.145 +/- 0.000
nu_eta                      0.120 +/- 0.000
lnubar_delta_eta                    0.117 +/- 0.000
bnu_delta_eta                    0.115 +/- 0.000
wplusb_delta_phi                    0.108 +/- 0.000
ttbar_mass                    0.094 +/- 0.000
wminus_eta                    0.089 +/- 0.000
lb_delta_eta                    0.081 +/- 0.000
l_eta                       0.077 +/- 0.000
wminusbbar_delta_eta                    0.076 +/- 0.000
bnubar_delta_eta                    0.063 +/- 0.000
lbbar_delta_eta                    0.061 +/- 0.000
b_eta                       0.059 +/- 0.0